In [3]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.parse

In [6]:
data = pd.read_csv('sample.csv', encoding='utf-8')

In [7]:
# 음식과 관광호텔을 제외한 행만 필터링
filtered_data = data[(data['KTO카테고리대분류명'] != '음식') & (data['KTO카테고리소분류명'] != '관광호텔')]

In [8]:
# 관광지명 추출
tourist_spots = filtered_data['관광지명'].tolist()

# 관광지명을 URL 인코딩하여 리스트에 저장
encoded_list = [urllib.parse.quote(s) for s in tourist_spots]

In [9]:
# 새로운 데이터프레임 생성
new_data = pd.DataFrame(columns=['시군구명', '관광지명', '숙박시설명', '숙박시설카테고리'])

# 중복된 숙박시설을 추적하기 위한 세트 생성
unique_accommodations = set()

# 숙박시설 정보 가져오기
for spot, row in zip(encoded_list, filtered_data.iterrows()):
    basic = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={spot}+%EA%B7%BC%EC%B2%98%20%EC%88%99%EB%B0%95'
    url = requests.get(basic, headers={'User-Agent': 'Mozilla/5.0'})
    html = url.content
    soup = BeautifulSoup(html, 'html.parser')

    # 숙박시설명 + 카테고리 추출
    span_name = soup.find_all(class_='YwYLL')
    span_category = soup.find_all(class_='YzBgS')

    # 각 숙박시설명과 카테고리를 묶어서 숙박시설 정보 생성
    for name, category in zip(span_name, span_category):
        accommodation_name = name.text.strip()
        accommodation_category = category.text.strip()
        
        # 중복된 숙박시설인지 확인하고 중복되지 않은 경우에만 데이터프레임에 추가
        if (row[1]['관광지명'], accommodation_name) not in unique_accommodations:
            new_data = pd.concat([new_data, pd.DataFrame({'시군구명': [row[1]['시군구명']],
                                                          '관광지명': [row[1]['관광지명']],
                                                          '숙박시설명': [accommodation_name],
                                                          '숙박시설카테고리': [accommodation_category]})], ignore_index=True)
            # 중복 제거 세트에 추가
            unique_accommodations.add((row[1]['관광지명'], accommodation_name))

# 새로운 데이터프레임 출력
print(new_data)

ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다', None, 10053, None)", ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다', None, 10053, None))